# Libraries

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

#### Color Printing

In [2]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

#print(color.BOLD + 'Hello World !' + color.END)

# Ignore Deprecation & pandas warnings
Solution 1: This warning is caused by numpy which deprecated the truth value check on empty array
It is impossible to take advantage of the fact that empty arrays are False, because an array can be False for other reasons.


Solution 2: https://stackoverflow.com/questions/40033471/getting-settingwithcopywarning-warning-even-after-using-loc-in-pandas



In [3]:
import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)

pd.options.mode.chained_assignment = None


### Label Encoding

In [4]:
# A class to encode appliances!
class LabelEncoder(object):

    def __init__(self):
        self.encoding = {}
        self.decoding = {}

    # Input: pandas dataframe and the column to encode, returns the a numpy array
    def apply_encoding(self, df):
        df.replace(self.encoding, inplace = True)
        return df.values
    
    # Input: the key, returns the decoded appliance
    def apply_decoding(self, key):
        return self.decoding[key]

    # Input: pandas dataframe and the column t create the uniques code per appliance! (e.g daskio)
    def create_encoding_decoding(self, df):
        custom_id = 1
        for appliance in df.unique():
            self.decoding[custom_id] = appliance
            self.encoding[appliance] = custom_id
            custom_id += 1

# data_process(dataset_path) 
#### performs the data input & aggelos' processing.
# Preparexy(dataframe)
#### performs data cleaning & preparing.

In [5]:
def preparexy(df, le):
    # Data Preparing
    # Encode labels with value 0-> n_classes -1
    
    features = ['I50', 'Φ50', 'I150', 'Φ150', 'I250', 'Φ250']
    
    # Features - X
    X = df[features]

    # Calculate Z-score in order to find outliers
    z = np.abs(stats.zscore(X))
    #print(z) # Visualize
    threshold = 2.5 # Change the threshold arbitrarily
    #print(np.where(z > threshold))
    df = df[(z < threshold).all(axis=1)] # Remove outliers that exceed the threshold given from dataset
    
    # Now get as X the 'clean' features
    X = df[features]

    # Scale the inputs (4 options; put comment in the ones you are not using)

    # -1- Standard Scaling
    #scaler = StandardScaler()
    #X = scaler.fit_transform(X)

    # -2- Min-max scaling 
    #scaler = preprocessing.MinMaxScaler()
    #X = scaler.fit_transform(X)

    # -3- Robust scaling
    scaler = preprocessing.RobustScaler()
    X = scaler.fit_transform(X)

    # -4- Normalizing
    #scaler = preprocessing.Normalizer()
    #X = scaler.fit_transform(X)

    # Label - Y
    y = le.apply_encoding(df['appliance'])   
    return(X,y)

def data_process(df):
    # Data Cleaning with Aggelos Rules for 50, 150, 250 phases
    df = df[(df.I50 > 0.1) & (df.I150 > 0.01) & (df.I250 > 0.01)] # Clean useless current features


    #df['Φ50'] = df['Φ50'].apply(pd.to_numeric)

    # For angle between (90, 180):
    # Modify by +180 degrees
    rows_with_rads_to_decrease = df.loc[(df['Φ50'] > 90) & (df['Φ50'] < 180)]
    rows_with_rads_to_decrease['Φ50'] -= 180
    df.update(rows_with_rads_to_decrease)

    # For angle between (-180, -90):
    # Modify by -180 degrees
    rows_with_rads_to_increase = (df.loc[(df['Φ50'] < -90) & (df['Φ50'] > -180)])
    rows_with_rads_to_increase['Φ50'] += 180
    df.update(rows_with_rads_to_increase)
    
    return df

### Load Datasets 
Following, you can either:
- train on house1, test on house2
- or split train/test set on houseX

In [6]:
# create an instance of label encoder!
le = LabelEncoder()

# Uncomment the following if you want to train on house1 and test on house2:
daskio = pd.read_excel("../datasets/appliances_combination_daskio.xls")
veroia = pd.read_excel("../datasets/appliances_combination_veroia.xls")

#init encoding and decoding from daskio dataset!
le.create_encoding_decoding(daskio['appliance'])
X_train, y_train = preparexy(data_process(daskio), le)
X_test, y_test = preparexy(data_process(veroia), le)

######----#####------######----#####------######----#####------######----#####------

# Train & Testing in 1 dataset only
#daskio = pd.read_excel("../datasets/appliances_combination_daskio.xls")

# init encoding and decoding from daskio dataset!
#le.create_encoding_decoding(daskio['appliance'])

# get x,y from daskio
#X, y = preparexy(data_process(daskio), le)

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## printMetrics method (Used when splitting train/test so you have a $X_{test}$ and a $y_{test}$)
Takes as input the $classifier$, the $X_{test}$ and the $y_{test}$.

Then it prints a detailed report with all the known metrics. 

In [7]:
def printMetrics(clf, X_test, y_test):
    # Uncomment if you want confusion matrix to be shown
    #cm = confusion_matrix(y_test, y_pred) 
    #print("Confusion matrix")
    #print(cm)
    
    
    # This will print precision, recall, f1-score, support for all the categories
    #target_names = ['class 0', 'class 1', 'class 2']
    print("Classification report for classifier \n%s:\n%s" % (clf, metrics.classification_report(y_test, y_pred)))
    print("Accuracy: %1.3f" % clf.score(X_test, y_test))
    print("-----------------\n")   

# get_metrics Method
### Takes as input the true class and the predicted class
#### Then, splits these classes into the labels containing each class and computes Precision, Recall, Support

In [8]:
# Compute the metrics mathematically 
def get_metrics(true_string, predicted_string):
    
    true_array = true_string.split("+") # Find substrings and split them
    predicted_array = predicted_string.split("+")
    
    # Find which elements of array A are in array B
    comparePositives = np.in1d(predicted_array, true_array)
    compareNegatives = np.in1d(true_array, predicted_array)

    #print(comparePositives)
    #print(compareNegatives)
    
    TP = TN = FP = FN = 0 # Initialize the metrics
    
    # Scan the arrays for each elements' presence
    for predictedLabel in comparePositives:
        if (predictedLabel == True):
            TP = TP + 1
        if (predictedLabel == False):
            FP = FP + 1
            
    for predictedLabel in compareNegatives:
        if (predictedLabel == True):
            TN = TN + 1
        if (predictedLabel == False):
            FN = FN + 1
            

    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    #print(precision)
    #print(recall)
    
    if (precision+recall == 0):
        F1 = 0
    else:
        F1 = 2*((precision*recall)/(precision+recall))
    
    
    #print (precision, recall, F1)
    
    return precision, recall, F1

#print(get_metrics("Laptop+mpriki+Tostiera", "DVD-TV+Mati"))
#print(get_metrics("DVD-TV+Pistolaki+Tostiera", "DVD-TV+Pistolaki"))
#DVD-TV+Psigio+Mati 
#DVD-TV+Mati
#print(get_metrics("DVD-TV+Psigio+Mati", "DVD-TV+Mati"))

## Decision Trees
##### Uncomment and then add the for-loop (2 cells below) in order to run it

In [18]:
# Decision Trees
clf = DecisionTreeClassifier(random_state = 42) # Feel free to change 'min_samples_split' 
clf.fit(X_train, y_train)

#print("Decision Trees:")
y_pred = clf.predict(X_test)

#printMetrics(clf, X_test, y_test)

## Multi-layer Perceptron with 18 neurons in 1 hidden layer

In [10]:
#print(X_test.shape, y_test.shape)
#print(le.inverse_transform(y_test[330])) # Check one TRUE class 

#true = le.inverse_transform(y_test[330])
#clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(18,), random_state=42)
#clf = LogisticRegression()
#clf.fit(X_train, y_train)

#print("Multi-layer Perceptron:")
#y_pred = clf.predict(X_test)
#print(le.inverse_transform(y_pred[330])) # check one PREDICTED class
#predicted = le.inverse_transform(y_pred[330])

#printMetrics(clf, X_test, y_test)

Multi-layer Perceptron:


### Initialize dictionary in which we will save the mean values

In [19]:
# Initialiaze a list
list_of_classes = []

# Populate the list with all the class names
for i in range(0, len(y_test)):
    true_class = le.apply_decoding(y_test[i])
    list_of_classes.append(true_class)

# Keep only non-duplicated class names
list_of_classes = sorted(set(list_of_classes), key=lambda x: list_of_classes.index(x))

# Initialiaze a dictionary thas has as 'key' the class name and as 'values' the precision, recall, f1 metrics
mean_dict = {class_name:[0, 0, 0, 0] for class_name in list_of_classes}

print(mean_dict)

{'Pistolaki+mpriki+Tostiera': [0, 0, 0, 0], 'DVD-TV+Laptop+Tostiera': [0, 0, 0, 0], 'DVD-TV+Tostiera': [0, 0, 0, 0], 'DVD-TV+Pistolaki+Tostiera': [0, 0, 0, 0], 'DVD-TV+Laptop+Pistolaki': [0, 0, 0, 0], 'DVD-TV+Mati': [0, 0, 0, 0], 'DVD-TV+Laptop+Mati': [0, 0, 0, 0], 'Laptop+mpriki+Tostiera': [0, 0, 0, 0], 'DVD-TV+Laptop': [0, 0, 0, 0], 'DVD-TV+Psigio': [0, 0, 0, 0], 'DVD-TV+Pistolaki+Mati': [0, 0, 0, 0], 'Laptop+Pistolaki+Tostiera': [0, 0, 0, 0], 'DVD-TV+Pistolaki': [0, 0, 0, 0]}


In [20]:
print(len(mean_dict))

13


In [21]:
for i in range(0, len(y_test)):

    true_class = le.apply_decoding(y_test[i])
    predicted_class = le.apply_decoding(y_pred[i])
    #print(true_class, predicted_class)
    
    #print(y_test[i], y_pred[i])
    #print('Actual class is {}{} and the classifier predicted {}'.format(color.BOLD, true_class, predicted_class))

    print('Actual class is ' + color.BOLD  + color.BLUE+ true_class + color.END +' and the classifier predicted ' + color.BOLD + color.RED + predicted_class + color.END)
    precision, recall, F1 = get_metrics(true_class, predicted_class)
    
    print('Precision is {}, Recall is {} and F1-Score is {}'.format(precision, recall, F1))
    
    precision = round(precision, 4)
    recall = round(recall, 4)
    F1 = round(F1, 4)
    
    # Add the values to the dictionary so we can have mean-values for the metrics
    mean_dict[true_class][0] = mean_dict[true_class][0] + precision 
    mean_dict[true_class][1] = mean_dict[true_class][1] + recall
    mean_dict[true_class][2] = mean_dict[true_class][2] + F1
    mean_dict[true_class][3] = mean_dict[true_class][3] + 1
    print('\n')
    
    
    

Actual class is Pistolaki+mpriki+Tostiera and the classifier predicted DVD-TV+Pistolaki+Tostiera
Precision is 0.6666666666666666, Recall is 0.6666666666666666 and F1-Score is 0.6666666666666666


Actual class is Pistolaki+mpriki+Tostiera and the classifier predicted DVD-TV+Pistolaki+Tostiera
Precision is 0.6666666666666666, Recall is 0.6666666666666666 and F1-Score is 0.6666666666666666


Actual class is Pistolaki+mpriki+Tostiera and the classifier predicted DVD-TV+Pistolaki+Tostiera
Precision is 0.6666666666666666, Recall is 0.6666666666666666 and F1-Score is 0.6666666666666666


Actual class is Pistolaki+mpriki+Tostiera and the classifier predicted DVD-TV+Pistolaki+Tostiera
Precision is 0.6666666666666666, Recall is 0.6666666666666666 and F1-Score is 0.6666666666666666


Actual class is Pistolaki+mpriki+Tostiera and the classifier predicted DVD-TV+Pistolaki+Tostiera
Precision is 0.6666666666666666, Recall is 0.6666666666666666 and F1-Score is 0.6666666666666666


Actual class is Pist

Actual class is DVD-TV+Laptop+Tostiera and the classifier predicted DVD-TV+Laptop+Psigio
Precision is 0.6666666666666666, Recall is 0.6666666666666666 and F1-Score is 0.6666666666666666


Actual class is DVD-TV+Laptop+Tostiera and the classifier predicted DVD-TV+Laptop+Psigio
Precision is 0.6666666666666666, Recall is 0.6666666666666666 and F1-Score is 0.6666666666666666


Actual class is DVD-TV+Laptop+Tostiera and the classifier predicted DVD-TV+Laptop+Psigio
Precision is 0.6666666666666666, Recall is 0.6666666666666666 and F1-Score is 0.6666666666666666


Actual class is DVD-TV+Laptop+Tostiera and the classifier predicted DVD-TV+Laptop+Psigio
Precision is 0.6666666666666666, Recall is 0.6666666666666666 and F1-Score is 0.6666666666666666


Actual class is DVD-TV+Laptop+Tostiera and the classifier predicted DVD-TV+Laptop+Psigio
Precision is 0.6666666666666666, Recall is 0.6666666666666666 and F1-Score is 0.6666666666666666


Actual class is DVD-TV+Laptop+Tostiera and the classifier pr

Actual class is DVD-TV+Tostiera and the classifier predicted DVD-TV+Tostiera
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


Actual class is DVD-TV+Tostiera and the classifier predicted Laptop+Psigio+Tostiera
Precision is 0.3333333333333333, Recall is 0.5 and F1-Score is 0.4


Actual class is DVD-TV+Tostiera and the classifier predicted Laptop+Psigio+Tostiera
Precision is 0.3333333333333333, Recall is 0.5 and F1-Score is 0.4


Actual class is DVD-TV+Tostiera and the classifier predicted DVD-TV+Laptop+Tostiera
Precision is 0.6666666666666666, Recall is 1.0 and F1-Score is 0.8


Actual class is DVD-TV+Tostiera and the classifier predicted DVD-TV+Laptop+Tostiera
Precision is 0.6666666666666666, Recall is 1.0 and F1-Score is 0.8


Actual class is DVD-TV+Tostiera and the classifier predicted DVD-TV+Laptop+Psigio
Precision is 0.3333333333333333, Recall is 0.5 and F1-Score is 0.4


Actual class is DVD-TV+Tostiera and the classifier predicted DVD-TV+Pistolaki+Mati
Precision is 0.33333333

Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


Actual class is DVD-TV+Pistolaki+Tostiera and the classifier predicted DVD-TV+Pistolaki+Mati
Precision is 0.6666666666666666, Recall is 0.6666666666666666 and F1-Score is 0.6666666666666666


Actual class is DVD-TV+Pistolaki+Tostiera and the classifier predicted DVD-TV+Pistolaki+Mati
Precision is 0.6666666666666666, Recall is 0.6666666666666666 and F1-Score is 0.6666666666666666


Actual class is DVD-TV+Pistolaki+Tostiera and the classifier predicted DVD-TV+Pistolaki+Tostiera
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


Actual class is DVD-TV+Pistolaki+Tostiera and the classifier predicted DVD-TV+Pistolaki+Mati
Precision is 0.6666666666666666, Recall is 0.6666666666666666 and F1-Score is 0.6666666666666666


Actual class is DVD-TV+Pistolaki+Tostiera and the classifier predicted DVD-TV+Pistolaki+Mati
Precision is 0.6666666666666666, Recall is 0.6666666666666666 and F1-Score is 0.6666666666666666


Actual class is DVD-TV+Pist

Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


Actual class is DVD-TV+Laptop+Pistolaki and the classifier predicted DVD-TV+Laptop+Pistolaki
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


Actual class is DVD-TV+Laptop+Pistolaki and the classifier predicted DVD-TV+Laptop+Pistolaki
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


Actual class is DVD-TV+Laptop+Pistolaki and the classifier predicted DVD-TV+Laptop+Pistolaki
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


Actual class is DVD-TV+Laptop+Pistolaki and the classifier predicted DVD-TV+Laptop+Pistolaki
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


Actual class is DVD-TV+Laptop+Pistolaki and the classifier predicted DVD-TV+Laptop+Pistolaki
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


Actual class is DVD-TV+Laptop+Pistolaki and the classifier predicted DVD-TV+Laptop+Pistolaki
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


Actual class is DVD-TV+Laptop+Pistolaki and the classifier predi

...

Actual class is DVD-TV+Psigio and the classifier predicted DVD-TV+Mati
Precision is 0.5, Recall is 0.5 and F1-Score is 0.5


Actual class is DVD-TV+Psigio and the classifier predicted DVD-TV+Mati
Precision is 0.5, Recall is 0.5 and F1-Score is 0.5


Actual class is DVD-TV+Psigio and the classifier predicted DVD-TV+Mati
Precision is 0.5, Recall is 0.5 and F1-Score is 0.5


Actual class is DVD-TV+Psigio and the classifier predicted DVD-TV+Pistolaki
Precision is 0.5, Recall is 0.5 and F1-Score is 0.5


Actual class is DVD-TV+Psigio and the classifier predicted DVD-TV+Pistolaki
Precision is 0.5, Recall is 0.5 and F1-Score is 0.5


Actual class is DVD-TV+Psigio and the classifier predicted DVD-TV+Pistolaki
Precision is 0.5, Recall is 0.5 and F1-Score is 0.5


Actual class is DVD-TV+Psigio and the classifier predicted DVD-TV+Mati
Precision is 0.5, Recall is 0.5 and F1-Score is 0.5


Actual class is DVD-TV+Psigio and the classifier predicted DVD-TV+Mati
Precision is 0.5, Recall is 0.5 and F1-

Precision is 0.6666666666666666, Recall is 0.6666666666666666 and F1-Score is 0.6666666666666666


Actual class is DVD-TV+Pistolaki+Mati and the classifier predicted DVD-TV+Pistolaki+Tostiera
Precision is 0.6666666666666666, Recall is 0.6666666666666666 and F1-Score is 0.6666666666666666


Actual class is DVD-TV+Pistolaki+Mati and the classifier predicted DVD-TV+Psigio+Pistolaki
Precision is 0.6666666666666666, Recall is 0.6666666666666666 and F1-Score is 0.6666666666666666


Actual class is DVD-TV+Pistolaki+Mati and the classifier predicted DVD-TV+Pistolaki+Mati
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


Actual class is DVD-TV+Pistolaki+Mati and the classifier predicted DVD-TV+Pistolaki
Precision is 1.0, Recall is 0.6666666666666666 and F1-Score is 0.8


Actual class is DVD-TV+Pistolaki+Mati and the classifier predicted DVD-TV+Psigio+Tostiera
Precision is 0.3333333333333333, Recall is 0.3333333333333333 and F1-Score is 0.3333333333333333


Actual class is DVD-TV+Pistolaki+M

Precision is 0.6666666666666666, Recall is 0.6666666666666666 and F1-Score is 0.6666666666666666


Actual class is Laptop+Pistolaki+Tostiera and the classifier predicted DVD-TV+Pistolaki+Tostiera
Precision is 0.6666666666666666, Recall is 0.6666666666666666 and F1-Score is 0.6666666666666666


Actual class is Laptop+Pistolaki+Tostiera and the classifier predicted DVD-TV+Pistolaki+Tostiera
Precision is 0.6666666666666666, Recall is 0.6666666666666666 and F1-Score is 0.6666666666666666


Actual class is Laptop+Pistolaki+Tostiera and the classifier predicted DVD-TV+Pistolaki+Tostiera
Precision is 0.6666666666666666, Recall is 0.6666666666666666 and F1-Score is 0.6666666666666666


Actual class is Laptop+Pistolaki+Tostiera and the classifier predicted DVD-TV+Pistolaki+Tostiera
Precision is 0.6666666666666666, Recall is 0.6666666666666666 and F1-Score is 0.6666666666666666


Actual class is Laptop+Pistolaki+Tostiera and the classifier predicted DVD-TV+Pistolaki+Tostiera
Precision is 0.66666



Actual class is DVD-TV+Laptop+Mati and the classifier predicted DVD-TV+Laptop+Mati
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


Actual class is DVD-TV+Laptop+Mati and the classifier predicted DVD-TV+Laptop+Mati
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


Actual class is DVD-TV+Laptop+Mati and the classifier predicted DVD-TV+Laptop+Mati
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


Actual class is DVD-TV+Laptop+Mati and the classifier predicted DVD-TV+Laptop+Mati
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


Actual class is DVD-TV+Laptop+Mati and the classifier predicted DVD-TV+Laptop+Mati
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


Actual class is DVD-TV+Laptop+Mati and the classifier predicted DVD-TV+Laptop+Mati
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


Actual class is DVD-TV+Laptop+Mati and the classifier predicted DVD-TV+Laptop+Mati
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


Actual class is DVD-TV+Laptop+Mati and 


Actual class is DVD-TV+Pistolaki and the classifier predicted DVD-TV+Psigio+Tostiera
Precision is 0.3333333333333333, Recall is 0.5 and F1-Score is 0.4


Actual class is DVD-TV+Pistolaki and the classifier predicted DVD-TV+Psigio+Tostiera
Precision is 0.3333333333333333, Recall is 0.5 and F1-Score is 0.4


Actual class is DVD-TV+Pistolaki and the classifier predicted DVD-TV+Laptop+Pistolaki
Precision is 0.6666666666666666, Recall is 1.0 and F1-Score is 0.8


Actual class is DVD-TV+Pistolaki and the classifier predicted DVD-TV+Laptop+Pistolaki
Precision is 0.6666666666666666, Recall is 1.0 and F1-Score is 0.8


Actual class is DVD-TV+Pistolaki and the classifier predicted Laptop+Psigio+Pistolaki
Precision is 0.3333333333333333, Recall is 0.5 and F1-Score is 0.4


Actual class is DVD-TV+Pistolaki and the classifier predicted Laptop+Psigio+Pistolaki
Precision is 0.3333333333333333, Recall is 0.5 and F1-Score is 0.4


Actual class is DVD-TV+Pistolaki and the classifier predicted Laptop+Ps

In [22]:
print(mean_dict)


{'Pistolaki+mpriki+Tostiera': [74.6643, 74.6643, 74.6643, 247], 'DVD-TV+Laptop+Tostiera': [456.01439999999724, 456.01439999999724, 456.01439999999724, 600], 'DVD-TV+Tostiera': [122.6614999999996, 171.5, 142.20000000000053, 286], 'DVD-TV+Pistolaki+Tostiera': [253.6678, 245.3352999999998, 248.66780000000028, 266], 'DVD-TV+Laptop+Pistolaki': [470.50609999999904, 467.33909999999906, 468.6060999999992, 553], 'DVD-TV+Mati': [144.6695000000001, 201.0, 167.20000000000036, 229], 'DVD-TV+Laptop+Mati': [599.3334, 599.3334, 599.3334, 600], 'Laptop+mpriki+Tostiera': [27.330600000000032, 27.330600000000032, 27.330600000000032, 248], 'DVD-TV+Laptop': [200.00999999999956, 300.0, 240.00000000000125, 300], 'DVD-TV+Psigio': [149.8333, 150.0, 149.9, 300], 'DVD-TV+Pistolaki+Mati': [203.83529999999996, 200.33529999999996, 201.73529999999994, 247], 'Laptop+Pistolaki+Tostiera': [127.16980000000014, 126.66970000000012, 126.86980000000015, 200], 'DVD-TV+Pistolaki': [200.67569999999947, 294.5, 238.20000000000113

In [23]:
for key in mean_dict:
    count = mean_dict[key][3]
    mean_dict[key][0] /= count
    mean_dict[key][1] /= count
    mean_dict[key][2] /= count
    
    

In [24]:
print(mean_dict)


{'Pistolaki+mpriki+Tostiera': [0.3022846153846154, 0.3022846153846154, 0.3022846153846154, 247], 'DVD-TV+Laptop+Tostiera': [0.7600239999999954, 0.7600239999999954, 0.7600239999999954, 600], 'DVD-TV+Tostiera': [0.42888636363636223, 0.5996503496503497, 0.49720279720279903, 286], 'DVD-TV+Pistolaki+Tostiera': [0.9536383458646617, 0.922313157894736, 0.9348413533834597, 266], 'DVD-TV+Laptop+Pistolaki': [0.8508247739602153, 0.8450978300180815, 0.847388969258588, 553], 'DVD-TV+Mati': [0.6317445414847167, 0.8777292576419214, 0.7301310043668138, 229], 'DVD-TV+Laptop+Mati': [0.998889, 0.998889, 0.998889, 600], 'Laptop+mpriki+Tostiera': [0.11020403225806465, 0.11020403225806465, 0.11020403225806465, 248], 'DVD-TV+Laptop': [0.6666999999999985, 1.0, 0.8000000000000042, 300], 'DVD-TV+Psigio': [0.4994443333333334, 0.5, 0.4996666666666667, 300], 'DVD-TV+Pistolaki+Mati': [0.8252441295546558, 0.8110740890688257, 0.8167421052631576, 247], 'Laptop+Pistolaki+Tostiera': [0.6358490000000007, 0.633348500000000

# Save the dictionary metrics to .csv file

In [26]:
output_df = pd.DataFrame(mean_dict)
output_df = output_df.T
output_df.to_csv('daskio_veroia_decision_trees.csv')
